In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# %% [code] cell 1: Install required libraries
!pip install transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00


In [3]:
# %% [code] cell 2: Login to Hugging Face
# This will prompt you to enter your HF token.
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# %% [code] cell 3: Upload your dataset (Algebra.jsonl)
# Run this cell and then select your Algebra.jsonl file from your local machine.
from google.colab import files
uploaded = files.upload()
# Ensure that "Algebra.jsonl" appears in the file list in the Colab file browser.

Saving Algebra.jsonl to Algebra.jsonl


In [5]:
# %% [code] cell 4: Import libraries and define helper functions
import os
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from collections import Counter

# --- Helper function to extract the answer from generated text ---
def extract_answer(text):
    # Assumes that the prompt is "Solve the following problem: <...>\nAnswer:"
    if "Answer:" in text:
        return text.split("Answer:")[-1].strip()
    else:
        return text.strip()

# --- Compute token-level F1 (similar to SQuAD F1) ---
def compute_f1_single(pred, gold):
    pred_tokens = pred.split()
    gold_tokens = gold.split()
    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

In [6]:
# %% [code] cell 5: Load and preprocess the dataset
# Here we assume your dataset file "Algebra.jsonl" is in the current directory.
data_files = {"test": "Algebra.jsonl"}
raw_dataset = load_dataset("json", data_files=data_files, split="test")

# Create new fields:
# - "input_text": a prompt created from the problem
# - "target": the ground truth answer (used later for metric calculation)
def preprocess_function(example):
    prompt = f"Solve the following problem: {example['problem']}\nAnswer:"
    return {"input_text": prompt, "target": example["answer"]}

processed_dataset = raw_dataset.map(preprocess_function)

# Save the ground-truth answers in a global variable for metric computation.
ground_truths = processed_dataset["target"]

# (Optional) Inspect one sample
print("Sample preprocessed example:")
print(processed_dataset[0])


Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Sample preprocessed example:
{'problem': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?', 'solution': 'The denominator of the rational function factors into $x^2+x-6=(x-2)(x+3)$. Since the numerator is always nonzero, there is a vertical asymptote whenever the denominator is $0$, which occurs for $x = 2$ and $x = -3$.  Therefore, the graph has $\\boxed{2}$ vertical asymptotes.', 'answer': '2', 'subject': 'Algebra', 'level': 3, 'unique_id': 'test/algebra/1.json', 'input_text': 'Solve the following problem: How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?\nAnswer:', 'target': '2'}


In [ ]:
# %% [code] cell 6: Tokenize the dataset
# We use a maximum length (e.g., 128 tokens) for the prompt.
hf_token = "YOUR TOKEN"  # Replace with your actual Hugging Face token
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

# Make sure the tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = processed_dataset.map(tokenize_function, batched=True)
# Remove columns not needed for model input (keep "input_ids", "attention_mask")
columns_to_remove = ["problem", "solution", "answer", "subject", "level", "unique_id", "input_text"]
tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)

print("Tokenized dataset example:")
print(tokenized_dataset[0])

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenized dataset example:
{'target': '2', 'input_ids': [128000, 50, 4035, 279, 2768, 3575, 25, 2650, 1690, 12414, 97354, 6429, 1587, 279, 4876, 315, 400, 88, 35533, 38118, 90, 17, 15523, 87, 61, 17, 38992, 12, 21, 32816, 617, 5380, 16533, 25, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001], 'at

In [8]:
# %% [code] cell 7: Define a custom Trainer that uses model.generate for zero-shot inference
class GenerationTrainer(Trainer):
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        # Get the input_ids and attention_mask from inputs
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        # Use model.generate to perform zero-shot inference.
        generated_tokens = model.generate(input_ids=input_ids,
                                          attention_mask=attention_mask,
                                          max_new_tokens=50)
        labels = inputs.get("labels")
        return (None, generated_tokens, labels)

# Define a compute_metrics function that uses our helper functions.
def compute_metrics(eval_preds):
    predictions, _ = eval_preds
    # Decode the generated tokens to strings.
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Extract the answer part from the generated text.
    pred_answers = [extract_answer(pred) for pred in decoded_preds]
    correct = 0
    f1s = []
    for pred_ans, gt in zip(pred_answers, ground_truths):
        if pred_ans.strip().lower() == gt.strip().lower():
            correct += 1
        f1s.append(compute_f1_single(pred_ans.lower(), gt.lower()))
    accuracy = correct / len(ground_truths)
    avg_f1 = np.mean(f1s)
    return {"accuracy": accuracy, "f1": avg_f1}

In [9]:
# %% [code] cell 8: Define TrainingArguments for evaluation (no training)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=2,
    do_predict=True,
    logging_steps=1,
    report_to="none"  # Turn off logging to third-party services
)

In [ ]:
# %% [code] cell 9: Create a function to load a model, evaluate it, and test interactively
def evaluate_model(model_name):
    print(f"\nEvaluating model: {model_name}")
    # Load the model (ensure you have access via your HF token)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Create our custom Trainer instance.
    trainer = GenerationTrainer(
        model=model,
        args=training_args,
        eval_dataset=tokenized_dataset,
        compute_metrics=compute_metrics
    )

    # Run evaluation on the tokenized dataset.
    metrics = trainer.predict(tokenized_dataset)
    print("Evaluation Metrics:", metrics.metrics)

    # Define an interactive test function for a new problem.
    def answer_question(problem):
        prompt = f"Solve the following problem: {problem}\nAnswer:"
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        generated = model.generate(input_ids=input_ids, max_new_tokens=50)
        output = tokenizer.decode(generated[0], skip_special_tokens=True)
        answer = extract_answer(output)
        return answer

    # For demonstration, test the model on the first problem in the dataset.
    test_problem = processed_dataset[0]["problem"]
    print("\n--- Interactive Test ---")
    print("Test Problem:")
    print(test_problem)
    print("Model Answer:")
    print(answer_question(test_problem))

    return trainer, metrics

# Evaluate Llama-3.2-1B
trainer_1B, metrics_1B = evaluate_model("meta-llama/Llama-3.2-1B")

# Evaluate Llama-3.2-3B
trainer_3B, metrics_3B = evaluate_model("meta-llama/Llama-3.2-3B")



Evaluating model: meta-llama/Llama-3.2-1B


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]